In [8]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By

from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import re
import configparser

In [9]:
#USER_LOGIN = '<YOUR LOGIN>'
#USER_PASSWORD = '<YOUR_PASSWORD>'

# загрузка данных из файла конфигурации
CFG_FILE = 'parser.ini'

# формат файла parser.ini
#[LINKEDIN]
#USER_LOGIN = эл_почта_без_кавычек
#USER_PASSWORD = пароль

# конфиг, файлы для парсинга
conf = configparser.ConfigParser()
try:
    conf.read(CFG_FILE)
    USER_LOGIN = conf['LINKEDIN']['USER_LOGIN']
    USER_PASSWORD = conf['LINKEDIN']['USER_PASSWORD']
except:
    logging.error(f'Не удалось прочитать файл конфигурации: {CFG_FILE}')
    quit() # нужен для файла .py

In [5]:
# получить и отобразить информацию профиля
def get_and_print_profile_info(driver, profile_url):
    driver.get(profile_url)        # открываем ссылку profile_url

    # извлекаем код страницы
    src = driver.page_source

    # предеаем код страницы в парсер
    soup = BeautifulSoup(src, 'lxml')

    # извлекаем HTML содержаший имя и заголовок
    intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

    #print(intro)

    # In case of an error, try changing the tags used here.
    name_loc = intro.find("h1")

    # Extracting the Name
    name = name_loc.get_text().strip()
    # strip() is used to remove any extra blank spaces

    works_at_loc = intro.find("div", {'class': 'text-body-medium'})

    # this gives us the HTML of the tag in which the Company Name is present
    # Extracting the Company Name
    works_at = works_at_loc.get_text().strip()

    print("Name -->",  name,
          "\nWorks At -->", works_at)

    POSTS_URL_SUFFIX = 'recent-activity/all/'

    time.sleep(0.5)

    # Get current url from browser
    cur_profile_url = driver.current_url
    print(cur_profile_url)

    # Parse posts
    get_and_print_user_posts(driver, cur_profile_url + POSTS_URL_SUFFIX)

In [6]:
def get_and_print_user_posts(driver, posts_url):
    driver.get(posts_url)

    #Simulate scrolling to capture all posts
    SCROLL_PAUSE_TIME = 1.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    # We can adjust this number to get more posts
    NUM_SCROLLS = 5

    for i in range(NUM_SCROLLS):
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # Parsing posts
    src = driver.page_source

    # Now using beautiful soup
    soup = BeautifulSoup(src, 'lxml')
    # soup.prettify()

    posts = soup.find_all('li', class_='profile-creator-shared-feed-update__container')
    # print(posts)

    print(f'Number of posts: {len(posts)}')
    for post_src in posts:
        post_text_div = post_src.find('div', {'class': 'feed-shared-update-v2__description-wrapper mr2'})

        # if post_text_div is None:
        #     print(post_src)

        if post_text_div is not None:
            post_text = post_text_div.find('span', {'dir': 'ltr'})
        else:
            post_text = None

        # If post text is found
        if post_text is not None:
            post_text = post_text.get_text().strip()
            print(f'Post text: {post_text}')

        reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__reactions-count'})

        # If number of reactions is written as text
        # It has different class name
        if reaction_cnt is None:
            reaction_cnt = post_src.find('span', {'class': 'social-details-social-counts__social-proof-text'})

        if reaction_cnt is not None:
            reaction_cnt = reaction_cnt.get_text().strip()
            print(f'Reactions: {reaction_cnt}')

    return

In [7]:
# параметры Chrome
caps = DesiredCapabilities().CHROME

# меняем стратегию - ждать, пока свойство document.readyState примет значение interactive
caps['pageLoadStrategy'] = 'eager'

# запускаем Chrome
driver = webdriver.Chrome()

# открываем страницу входа linkedIn
# ВНИМАНИЕ: необходимо отключить двухфакторную аутонтификацию
driver.get("https://linkedin.com/uas/login")

# ожидаем загрузку страницы
time.sleep(3.5)

# поле ввода имени пользователя
username = driver.find_element(By.ID, "username")
# в случае ошибки попробуйте изменить используемый здесь тег элемента

# вводим свой Email
username.send_keys(USER_LOGIN)

# поле ввода пароля
pword = driver.find_element(By.ID, "password")
# в случае ошибки попробуйте изменить используемый здесь тег элемента

# вводим пароль
pword.send_keys(USER_PASSWORD)

# нажимаем кнопку Войти
# Формат (синтаксис) написания XPath --> //tagname[@attribute='value']
driver.find_element(By.XPATH, "//button[@type='submit']").click()

### выполнит тестовый парсинг страницы
# get_and_print_profile_info(driver, 'https://www.linkedin.com/in/mathurin-ache-11004218')
get_and_print_profile_info(driver, 'https://www.linkedin.com/in/yaninaav/')

exit()

# Open search page
driver.get('https://www.linkedin.com/search/results/people/?keywords=data%20scientist&origin=CLUSTER_EXPANSION&sid=1gy')

profile_urls = []

NUM_PAGES_TO_PARSE = 12

# Iterate over pages of search results
# to collect profile urls
for i in range(NUM_PAGES_TO_PARSE):
    search_result_links = driver.find_elements(By.CSS_SELECTOR, "div.entity-result__item a.app-aware-link")

    for link in search_result_links:
        href = link.get_attribute("href")
        if 'linkedin.com/in' in href:
            profile_urls.append(href)

    next_button = driver.find_element(By.CLASS_NAME,'artdeco-pagination__button--next')
    next_button.click()
    time.sleep(2)


profile_urls = list(set(profile_urls))

print(profile_urls)

# Parse profile urls
for profile_url in profile_urls:
    get_and_print_profile_info(driver, profile_url)
    time.sleep(2)

# close the Chrome browser
driver.quit()

WebDriverException: Message: unknown error: net::ERR_CONNECTION_RESET
  (Session info: chrome=114.0.5735.134)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x0075A813+48355]
	(No symbol) [0x006EC4B1]
	(No symbol) [0x005F5358]
	(No symbol) [0x005F1B33]
	(No symbol) [0x005E8CA5]
	(No symbol) [0x005E9D0A]
	(No symbol) [0x005E9003]
	(No symbol) [0x005E8355]
	(No symbol) [0x005E82EC]
	(No symbol) [0x005E6F26]
	(No symbol) [0x005E7688]
	(No symbol) [0x005F6B52]
	(No symbol) [0x0064CF01]
	(No symbol) [0x0063A73C]
	(No symbol) [0x0064C922]
	(No symbol) [0x0063A536]
	(No symbol) [0x006182DC]
	(No symbol) [0x006193DD]
	GetHandleVerifier [0x009BAABD+2539405]
	GetHandleVerifier [0x009FA78F+2800735]
	GetHandleVerifier [0x009F456C+2775612]
	GetHandleVerifier [0x007E51E0+616112]
	(No symbol) [0x006F5F8C]
	(No symbol) [0x006F2328]
	(No symbol) [0x006F240B]
	(No symbol) [0x006E4FF7]
	BaseThreadInitThunk [0x769D7D59+25]
	RtlInitializeExceptionChain [0x7788B74B+107]
	RtlClearBits [0x7788B6CF+191]
